# Init & Login

In [1]:
from math import ceil
from random import randrange
from time import sleep

import spotipy
import spotipy.util as util
from IPython.display import clear_output

## Client Info ##
CLIENT_ID     = ""
CLIENT_SECRET = ""
CLIENT_SCOPE  = "user-follow-modify playlist-modify-private playlist-modify-public"
USER_NAME     = "31ytgsr7wdmiaroy77msqpiupdsi"
REDIR_URI     = "https://github.com/jwatson-CO-edu/yt_shuffle_so_good"
AUTH_URL      = 'https://accounts.spotify.com/api/token'
BASE_URL      = 'https://api.spotify.com/v1/'
## API Info ##
_RESPONSE_LIMIT = 100

with open( "../keys/spot_ID.txt" , 'r' ) as f:
    CLIENT_ID = f.readlines()[0].strip()

with open( "../keys/spot_SECRET.txt" , 'r' ) as f:
    CLIENT_SECRET = f.readlines()[0].strip()

token = util.prompt_for_user_token(
    username      = USER_NAME,
    scope         = CLIENT_SCOPE,
    client_id     = CLIENT_ID,
    client_secret = CLIENT_SECRET,
    redirect_uri  = REDIR_URI
)

spot = spotipy.Spotify( auth = token )
clear_output( wait = False )

# Playlist Functions

In [4]:

def get_playlist_length( playlist_ID ):
    """ Get the number of total tracks in the playlist """
    response = spot.user_playlist_tracks(
        CLIENT_ID, 
        playlist_ID, 
        fields = 'items,uri,name,id,total', 
        limit  = _RESPONSE_LIMIT
    )
    return response['total']


def reorder_playlist_by_chunks( playlist_ID, chunkSize, Nmoves = None, verbose = False ):
    """ Shuffle the playlist a chunk at a time """
    Ntracks = get_playlist_length( playlist_ID )
    Nchunks = int( ceil( Ntracks / chunkSize ) )
    if Nmoves is None:
        Nmoves = int( Ntracks / chunkSize )
    for i in range( Nmoves ):
        bgnDex   = randrange(0,Nchunks) * chunkSize
        moveSize = min( chunkSize, Ntracks - bgnDex - 1 )
        sendDex  = randrange(0,Ntracks)
        try:
            response = spot.playlist_reorder_items(
                playlist_ID, 
                bgnDex, 
                sendDex, 
                range_length = chunkSize
            )
            if verbose:
                print( f"Iteration {i+1}, {bgnDex}-{bgnDex+moveSize} --> {sendDex}:\n\t{response}" )
        except spotipy.SpotifyException as ex:
            if verbose:
                print( f"ERROR, {bgnDex}-{bgnDex+moveSize} --> {sendDex}:\n\t{ex}" )


def complete_reorder_entire_playlist( playlist_ID, verbose = False, phasePause_s = 5 ):
    """ Completely shuffle the playlist in decreasing chunks """
    Ntracks = get_playlist_length( playlist_ID )
    ## Phase 1: Chunks of 100 ##
    if verbose:
        print("\nPhase 1: Chunks of 100\n")
    reorder_playlist_by_chunks( playlist_ID, 100, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 2: Chunks of 10 ##
    if verbose:
        print("\nPhase 2: Chunks of 10\n")
    reorder_playlist_by_chunks( playlist_ID, 10, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 3: Chunks of 5, 1/2 ##
    if verbose:
        print("\nPhase 3: Chunks of 5, 1/2\n")
    Nmoves = int( ceil( Ntracks / 10 ) )
    reorder_playlist_by_chunks( playlist_ID, 5, Nmoves, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 3: 1 at a Time, 1/3 ##
    if verbose:
        print("\nPhase 3: 1 at a Time, 1/3\n")
    Nmoves = int( ceil( Ntracks / 3 ) )
    reorder_playlist_by_chunks( playlist_ID, 1, Nmoves, verbose = verbose )

# Shuffle Playlists!

In [5]:
playlist = {
    'study01' : "0a2qoe6S7lYeZ6nlhZdA0v",
    'study02' : "6gbtR2cBq5PvkghidCvvGk",
    'study03' : "3o3lN2qntdEV7UKTuuC77K",
    'study04' : "41sFSisljvBDMBXtpp5NIw",
    'study05' : "02iS5AFGp8YVuUUqcQf8ys",
    'study06' : "6KI7A4MWrSM7EyKRUjxIi1",
    'study07' : "3V055Md2JdrUT8tX0af7di",
    'study08' : "0tspdJlwSgiyf2O9PO6QaP",
    'study09' : "5mHRBFoQtYy2izeZ66pG95",
    'study10' : "3832xeKGEOAXFJqE4K8kIq",
    'study11' : "65MXR4dubPL9t0P4dgTWvn",
    'cringe'  : "2AAUYlKM1nXKHkZUpUSFbv",
}

complete_reorder_entire_playlist( playlist['cringe'], verbose = True, phasePause_s = 5 )


Phase 1: Chunks of 100

Iteration 1, 200-300 --> 178:
	{'snapshot_id': 'ODg1LDk1MzE0ZjgxM2YzOWJjZTU3NDMzMmQxM2MwMmQxNGVjNmM0YWM4YjE='}
Iteration 2, 0-100 --> 150:
	{'snapshot_id': 'ODg2LDMzNzM2YTFkNTM5Mjg3ODZhNzZmMGM2ZWEwYmJkYWMzY2FhOTcxOTU='}
Iteration 3, 0-100 --> 149:
	{'snapshot_id': 'ODg3LDAxYmI3ZjhkMDJkNTVmYzU1MDA3YzE3YWZjNDZkYzI0YTdkMDU2Zjk='}

Phase 2: Chunks of 10

Iteration 1, 80-90 --> 151:
	{'snapshot_id': 'ODg4LDAxNTIyODViM2UwY2VkMzcxZGE2ZTJjMmQ0OWRmNDRmM2M4YzVmYzY='}
Iteration 2, 20-30 --> 28:
	{'snapshot_id': 'ODg5LDU1NDczMzUwMjI4MWM3MDUwYmVlMGUxMmUyOGY4M2MyMzc2ZTc5OGI='}
Iteration 3, 170-180 --> 127:
	{'snapshot_id': 'ODkwLGVhZDI0Njg1ZDdhNzIyY2JmZDY4MGM4MDQ3YzYyNjE2M2JlMmNlNzM='}
Iteration 4, 140-150 --> 47:
	{'snapshot_id': 'ODkxLDFiNzRlZTFhZGQ3NDY2YmI0YzJjMzFmYmJiNjliMzgxM2YzMjUyOTc='}
Iteration 5, 110-120 --> 164:
	{'snapshot_id': 'ODkyLDM1NDJhOTQxZTRhMTI1ZWEwYmQ2MjFlMjQ3MTczODZhN2ExZmRjNDI='}
Iteration 6, 50-60 --> 3:
	{'snapshot_id': 'ODkzLDlmN2Y1ZDVlMjkxMjE3MDY4M

HTTP Error for PUT to https://api.spotify.com/v1/playlists/2AAUYlKM1nXKHkZUpUSFbv/tracks with Params: {} returned 400 due to Tracks selected to be reordered are out of bounds


ERROR, 305-305 --> 87:
	http status: 400, code:-1 - https://api.spotify.com/v1/playlists/2AAUYlKM1nXKHkZUpUSFbv/tracks:
 Tracks selected to be reordered are out of bounds, reason: None
Iteration 7, 175-180 --> 148:
	{'snapshot_id': 'OTIzLDU0MDIyNzViMzVmNmY1OTg5MmU1ODJmMGNmZGFlMTg4YjRmMzRkN2I='}
Iteration 8, 135-140 --> 255:
	{'snapshot_id': 'OTI0LGU0ZTNiOGRkMGVlYzg2NWMzM2IxODNiODFjNDFiMGE0YjAxOTc1Njc='}
Iteration 9, 210-215 --> 203:
	{'snapshot_id': 'OTI1LDU3YTdiODA4NjExMWIxZWMyODA3NTEzMGNhYzQyMzdjODYyMDc4OTE='}
Iteration 10, 260-265 --> 218:
	{'snapshot_id': 'OTI2LGRkNGZjYjFjMzVhYmQ5Y2Y0Yzg4MmFlMjA1MjkzMTA5NTNiNDNmZTA='}
Iteration 11, 5-10 --> 194:
	{'snapshot_id': 'OTI3LDk1MGEwMDZkZTRlODA2NTU1YmEyYmEwMWU3OTI4MWQ2NTExZjMxZTk='}
Iteration 12, 195-200 --> 257:
	{'snapshot_id': 'OTI4LDBkZTE2NWIwYzNmOTMxOGY0ZTBiZjcwYjlkZWMyMmRmZTE0NjhkODk='}
Iteration 13, 30-35 --> 234:
	{'snapshot_id': 'OTI5LGU3NGYzMmNlOTI4OGMxN2RjNWViZTc2ZTg4YjAwZDI5NjJlOWJhZjU='}
Iteration 14, 130-135 --> 282:
	{'snaps

HTTP Error for PUT to https://api.spotify.com/v1/playlists/2AAUYlKM1nXKHkZUpUSFbv/tracks with Params: {} returned 400 due to Tracks selected to be reordered are out of bounds


Iteration 24, 145-150 --> 45:
	{'snapshot_id': 'OTQwLGM1NDZmNjMzOWE0MDBjN2MyZmJjMGQwMTE3OWViN2ZkY2UwYzY2MmE='}
ERROR, 305-305 --> 33:
	http status: 400, code:-1 - https://api.spotify.com/v1/playlists/2AAUYlKM1nXKHkZUpUSFbv/tracks:
 Tracks selected to be reordered are out of bounds, reason: None
Iteration 26, 200-205 --> 262:
	{'snapshot_id': 'OTQxLGVhMjE1Y2MwOWUwZmRjYjIzZjJmNzZlMGRlNTRkMzc2OGNiYjU4MTU='}
Iteration 27, 235-240 --> 263:
	{'snapshot_id': 'OTQyLDA4OWUyMDAwNDUyNzU5OWM0YzQ1YmJlYzc2YTVkYWJjOWMxZTMzYWE='}
Iteration 28, 275-280 --> 231:
	{'snapshot_id': 'OTQzLDFmMTliZTk5MTdiZDE0Y2E1N2Q3YTIwZjdjMjFmMzFhMGY1YTU2NjA='}
Iteration 29, 225-230 --> 128:
	{'snapshot_id': 'OTQ0LDhkOGNjMjNjNjIyMTI4OGE5N2Q1ZTg1NzI3YWZhMjY0MDM1YTBiMGE='}
Iteration 30, 255-260 --> 264:
	{'snapshot_id': 'OTQ1LGYxOTdiMTIzMDRjY2U0MmFiOWE4YTJlZmMxYjA1MjYyZGQwOGZjMjQ='}
Iteration 31, 190-195 --> 286:
	{'snapshot_id': 'OTQ2LDdhZjJlZmMwNTMwNmE5ZTk5OWNiNDgwNWMyMzNmNWM1MjYwNzJkMjQ='}

Phase 3: 1 at a Time, 1/3

Iter